## Imports
----

In [2]:
import logging
import time
import math
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import plotnine as gg
from scipy import stats
from datetime import datetime
import bayesflow as bf

from prl_utils import (
    Mode,
    get_features,
    read_hdf5,
    get_labels,
    padding,
    normalize_train_labels,
    normalize_val_labels,
    get_recovered_parameters,
    plot_recovery,
)


ModuleNotFoundError: No module named 'bayesflow'

## Read in data
-----

In [ ]:
# @title Constants
DRIVE_DIR = '/content/gdrive/MyDrive/dl4rl'
N_TRAIN_AGENT = 3000
N_VAL_AGENT = 3000
NUM_TRIAL = 2000

mode = Mode.PRL2_intractable
model_type = "GRU" #@@param ["GRU", "LSTM", "Transformer"] {allow-input: true}

In [ ]:
# @title load features
from numpy import load

prefix = f'data/vara_{N_TRAIN_AGENT}agent_{NUM_TRIAL}t_2ParamRL_intractable'
# load dict of arrays
train_features = load(f'{prefix}_features.npz')['arr_0']
train_labels = load(f'{prefix}_pest_labels.npz', allow_pickle=True)['arr_0'].tolist()
normalized_train_labels, name_to_scaler = normalize_train_labels(train_labels)

prefix = f'data/vara_{N_VAL_AGENT}agent_{NUM_TRIAL}t_2ParamRL_intractable'
val_features = load(f'{prefix}_features_val.npz')['arr_0']
val_labels = load(f'{prefix}_pest_labels_val.npz', allow_pickle=True)['arr_0'].tolist()
normalized_val_labels = normalize_val_labels(val_labels, name_to_scaler)

print(train_features.shape, len(train_labels))
OUPUT_DIM = len(train_labels)

In [ ]:
# @title normalization helper functions
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize(names_to_labels: dict, full_range_labels: dict):
  names = list(names_to_labels.keys())
  names.sort()

  normalized_labels = []
  name_to_scaler = {}
  for name in names:
    scaler = StandardScaler()
    scaler.fit(full_range_labels[name])
    normalized_labels.append(scaler.transform(names_to_labels[name]))
    name_to_scaler[name] = scaler

  return np.concatenate(normalized_labels, axis=-1), name_to_scaler

def mx_normalize(names_to_labels: dict):
  names = list(names_to_labels.keys())
  names.sort()

  normalized_labels = []
  name_to_scaler = {}
  for name in names:
    mmscaler = MinMaxScaler()
    if name == 'alpha':
      data_min = 0
      data_max = 1
    elif name == 'beta':
      data_min = 0
      data_max = 10

    # Update with custom range
    data_range = data_max - data_min
    scale_ = (1 - 0) / data_range
    mmscaler.scale_ = scale_
    mmscaler.min_ = 0 - data_min * scale_

    normalized_labels.append(mmscaler.transform(names_to_labels[name]))
    name_to_scaler[name] = mmscaler

  return np.concatenate(normalized_labels, axis=-1), name_to_scaler

In [ ]:
# @title load features
from numpy import load

prefix = f'{RL_DRIVE_DIR}{DATA_DIR}/with_states/vara_{N_TRAIN_AGENT}agent_{NUM_TRIAL}t_2ParamRL_intractable'
# load dict of arrays
train_features = load(f'{prefix}_features.npz')['arr_0']
train_labels = load(f'{prefix}_pest_labels.npz', allow_pickle=True)['arr_0'].tolist()
normalized_train_labels, name_to_scaler = normalize_train_labels(train_labels)

prefix = f'{RL_DRIVE_DIR}{DATA_DIR}/with_states/vara_{N_VAL_AGENT}agent_{NUM_TRIAL}t_2ParamRL_intractable'
val_features = load(f'{prefix}_features_val.npz')['arr_0']
val_labels = load(f'{prefix}_pest_labels_val.npz', allow_pickle=True)['arr_0'].tolist()
normalized_val_labels = normalize_val_labels(val_labels, name_to_scaler)

print(train_features.shape, len(train_labels))
OUPUT_DIM = len(train_labels)

In [ ]:
# @title Process data
from prl_utils import get_mixed_trials_data

target_trial = 2000
all_trials = [target_trial]
train_features = all_train_features[:, :target_trial, :] #get_mixed_trials_data(all_train_features, all_trials, mode=mode)
val_features = all_val_features[:, :target_trial, :] #get_mixed_trials_data(all_val_features, all_trials, mode=mode)

# normalize labels
normalized_train_labels, name_to_scaler = normalize_train_labels(train_name_to_labels)
normalized_val_labels = normalize_val_labels(val_name_to_labels, name_to_scaler)

print(train_features.shape, len(val_name_to_labels))
OUPUT_DIM = len(val_name_to_labels)

## Model Training

### BayesFlow

In [ ]:
from sklearn.utils import shuffle

train_features, normalized_train_labels = shuffle(train_features.numpy(), normalized_train_labels, random_state=0)
val_features, normalized_val_labels = shuffle(val_features.numpy(), normalized_val_labels, random_state=0)

In [ ]:
import bayesflow as bf

lstm_units= 36
summary_dim= 10
num_train_agent = 20000
batch_size = 128

identifier = f'bi{target_trial}t_L{lstm_units}_S{summary_dim}_B{batch_size}_at{num_train_agent}'
#simulations_dict sim_data and prior_draws to be present
test_inp = {'sim_data': train_features[:num_train_agent], 'prior_draws': normalized_train_labels[:num_train_agent]}
summary_net = bf.networks.SequenceNetwork(lstm_units=lstm_units, summary_dim=summary_dim, bidirectional=True)
inference_net = bf.networks.InvertibleNetwork(
    num_params=normalized_train_labels.shape[-1],
    num_coupling_layers=4,
    coupling_settings={"dense_args": dict(kernel_regularizer=None), "dropout": False},
)

# summary_rep = summary_net(test_inp["sim_data"]).numpy()
# print("Shape of simulated data sets: ", test_inp["sim_data"].shape)
# print("Shape of summary vectors: ", summary_rep.shape)
# z, log_det_J = inference_net(test_inp["prior_draws"], summary_rep)
# print("Shape of latent variables:", z.numpy().shape)
# print("Shape of log det Jacobian:", log_det_J.numpy().shape)

amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net)
trainer = bf.trainers.Trainer(amortizer=amortizer, default_lr=3e-4, checkpoint_path=TEST_RESULT_DIR+'/'+identifier)

In [ ]:
history = trainer.train_offline(
    simulations_dict=test_inp,
    epochs=200,
    batch_size=batch_size,
    early_stopping=True,
    validation_sims={'sim_data': val_features[:1000], 'prior_draws': normalized_val_labels[:1000]},
)

fig = bf.diagnostics.plot_losses(history["train_losses"], history["val_losses"])
fig.savefig(f'{TEST_RESULT_DIR}/{identifier}_loss.png')

### Our vanilla approach

In [ ]:
# @title Model definition

from tensorflow.keras import layers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.layers import (
    Dense,
    Dropout,
    LSTM,
    Bidirectional,
    GRU,
)
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD

tf.keras.utils.set_random_seed(33)
tf.config.experimental.enable_op_determinism()

# Define the LSTM model
def create_lstm_model(input_x: int, input_y: int, units: int=70, dropout: float=0.2, dropout1: float=0.2, dropout2: float=0.2, learning_rate: float=1e-3):
    activation_func = 'relu'

    model = keras.Sequential()
    model.add(Bidirectional(LSTM(units, return_sequences=False, input_shape=(input_x, input_y))))
    model.add(Dropout(dropout))
    model.add(Dense(int(units/2), activation=activation_func))
    model.add(Dropout(dropout1))
    model.add(Dense(int(units/4), activation=activation_func))
    model.add(Dropout(dropout2))
    #model.add(Dense(10, activation=activation_func))
    model.add(Dense(OUPUT_DIM, activation='linear'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

# Define the LSTM model
def create_stacked_lstm_model(input_x: int, input_y: int, units: int, dropout: float, dropout1: float, dropout2: float, learning_rate: float):
    activation_func = 'relu'

    model = keras.Sequential()
    model.add(Bidirectional(LSTM(units=units, return_sequences=True, input_shape=(input_x, input_y))))
    model.add(Dropout(dropout))
    model.add(Bidirectional(LSTM(units=int(units/2))))
    model.add(Dropout(dropout1))
    model.add(Dense(units=int(units/4), activation=activation_func))
    model.add(Dropout(dropout2))
    #model.add(Dense(10, activation=activation_func)) # not in 4rpl
    model.add(Dense(units=OUPUT_DIM, activation='linear'))

    optimizer = Adam(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

def create_gru_model(input_x: int, input_y: int, units: int=70, dropout: float=0.2, dropout1: float=0.2, dropout2: float=0.1, learning_rate: float=3e-4, m: Mode=Mode.PRL4):
    activation_func = 'relu'
    init_scheme = keras.initializers.HeNormal(seed=666)

    model = keras.Sequential([layers.Masking(mask_value=-1., input_shape=(input_x, input_y)), GRU(units, return_sequences=False)])
    model.add(Dropout(dropout))
    model.add(Dense(int(units/2), activation=activation_func, kernel_initializer=init_scheme))
    model.add(Dropout(dropout1))
    model.add(Dense(int(units/4), activation=activation_func, kernel_initializer=init_scheme))
    model.add(Dropout(dropout2))
    #model.add(Dense(10, activation=activation_func))
    model.add(Dense(OUPUT_DIM, activation='linear', kernel_initializer=init_scheme))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

def create_transformer_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
    learning_rate=0.01,
):
    def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(x, x)
        x = layers.Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(OUPUT_DIM, activation="linear")(x)
    model = keras.Model(inputs, outputs)

    # key_dim = head_size
    # lr = CustomSchedule(key_dim)
    # optimizer = tf.keras.optimizers.Adam(lr, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    model.compile(
      loss="mse",
      optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    )
    return model

def get_model(parms):
    return create_lstm_model(
        parms['input_x'], parms['input_y'], parms['units'], parms['dropout'], parms['dropout1'], parms['dropout2'], parms['learning_rate'])

def get_gru_model(parms):
    return create_gru_model(
        parms['input_x'], parms['input_y'], parms['units'], parms['dropout'], parms['dropout1'], parms['dropout2'], parms['learning_rate'])

def get_stacked_bi_model(parms):
    return create_stacked_lstm_model(
        parms['input_x'], parms['input_y'], parms['units'], parms['dropout'], parms['dropout1'], parms['dropout2'], parms['learning_rate'])

def get_transformer_model(parms):
    return create_transformer_model(
        parms['input_shape'],
        parms['head_size'],
        parms['num_head'],
        parms['ff_dim'],
        parms['num_transformer_blocks'],
        [parms['mlp_units']],
        parms['dropout'],
        parms['mlp_dropout'],
        parms['learning_rate'],
    )

In [ ]:
if model_type == 'Transformer':
  units = 186
  dropout = 0.3
  mlp_dropout = 0.4
  learning_rate = 0.003
  head_size = 103
  num_heads = 2
  batch_size = 128
  ff_dim = 4 # 10
  identifier = f'2PRL_intra_60k_T_H{num_heads}_FD_{ff_dim}_D{dropout}_MD{mlp_dropout}_{learning_rate}'

  best_model = create_transformer_model(
      train_features.shape[1:],
      head_size=head_size,
      num_heads=num_heads,
      ff_dim=ff_dim,
      num_transformer_blocks=2,
      mlp_units=[units],
      mlp_dropout=mlp_dropout,
      dropout=dropout,
      learning_rate=learning_rate
  )
else:
  batch_size = 256
  units = 256 #
  dropout = 0.2
  dropout1 = 0.1
  dropout2 = 0.02
  learning_rate = 3e-4

  identifier = f'vara_B{batch_size}_U{units}_D{dropout}_D{dropout1}_D{dropout2}_{learning_rate}'
  best_model = create_gru_model(
      input_x=train_features.shape[1],
      input_y=train_features.shape[2],
      units=units,
      dropout=dropout,
      dropout1=dropout1,
      dropout2=dropout2,
      learning_rate=learning_rate)
identifier

In [ ]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.001)
callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]

history = best_model.fit(
      train_features,
      normalized_train_labels,
      epochs=200,
      batch_size=batch_size,
      callbacks=callbacks,
      #validation_split=0.1,
      validation_data = (val_features, normalized_val_labels),
      verbose=2)

In [ ]:
from prl_utils import (
    plot_loss,
)
plot_loss(history, "", "")#f"{TEST_RESULT_DIR}/{identifier}_loss_plot.png"

In [ ]:
best_model.save(f'{TEST_RESULT_DIR}/{identifier}_model')

#Load model data
# best_model = tf.keras.models.load_model(f'{TEST_RESULT_DIR}/500t_B256_U128_D0.2_D0.1_D0.01_0.0003_model')
# best_model.summary()

## Model Evaluation

In [ ]:
# @title load from test features
from numpy import load

prefix = f'{RL_DRIVE_DIR}{DATA_DIR}/with_states/vara_{N_VAL_AGENT}agent_{NUM_TRIAL}t_2ParamRL_intractable'
# load dict of arrays
features = load(f'{prefix}_features_test.npz')
test_features = features['arr_0']

labels = load(f'{prefix}_pest_labels_test.npz', allow_pickle=True)
test_labels = labels['arr_0'].tolist()
normalized_test_labels = normalize_val_labels(test_labels, name_to_scaler)

In [ ]:
# @title load from test data
dist_name = '' #'_betadistribution_inrange' # _betadistribution
#test_data = pd.read_csv(f'{RL_DRIVE_DIR}{DATA_DIR}/{N_VAL_AGENT}agent_{NUM_TRIAL}t_{PARAM}_test.csv')
test_data = pd.read_csv(f'{RL_DRIVE_DIR}{DATA_DIR}/with_states/{N_VAL_AGENT}agent_{NUM_TRIAL}t_{PARAM}_test.csv')
#test_features = get_test_features_by_trials(test_data, N_VAL_AGENT, NUM_TRIAL, [500], mode=mode)
test_features = get_features(test_data, N_VAL_AGENT, NUM_TRIAL, mode=mode)
test_name_to_labels = get_labels(test_data, mode)
normalized_test_labels = normalize_val_labels(test_name_to_labels, name_to_scaler)

print(test_features.shape, normalized_test_labels.shape)

In [ ]:
# @title Bayesflow evalutaion
#test_sims = trainer.configurator({'sim_data': test_features, 'prior_draws': normalized_test_labels})
test_sims = trainer.configurator({'sim_data': test_features[:1000, :target_trial, :], 'prior_draws': normalized_test_labels[:1000]})
posterior_draws = amortizer.sample(test_sims, n_samples=1000)

f = bf.diagnostics.plot_recovery(
    posterior_draws, test_sims["parameters"], param_names=['T', 'alpha', 'beta'], point_agg=np.mean, uncertainty_agg=np.std
)
f.savefig(f'{TEST_RESULT_DIR}/{identifier}_recovery.png')

In [ ]:
# @title vanilla model: Compute the prediction
all_prediction = best_model.predict(test_features)

In [ ]:
recovered = get_recovered_parameters(name_to_scaler, test_labels, all_prediction)

In [ ]:
label = list(test_labels.keys())[0]
p = plot_recovery(recovered, label)
p.draw()

### Varied trials evaluation

In [ ]:
start = N_VAL_AGENT
trial_to_param_all_test = {'num_test_trial': []}
for i, num_trial in zip(range(start, start*len(ALL_TRIALS)+1, start), ALL_TRIALS):
  prediction = all_prediction[i-start:i, :]
  recovered = get_recovered_parameters(name_to_scaler, test_name_to_labels, prediction)
  trial_to_param_all_test['num_test_trial'].extend([num_trial]*N_VAL_AGENT)
  for col in recovered.columns:
    if col in trial_to_param_all_test:
      trial_to_param_all_test[col].extend(list(recovered[col]))
    else:
      trial_to_param_all_test[col] = list(recovered[col])

trial_to_param_all_test = pd.DataFrame(trial_to_param_all_test)

In [ ]:
trial_to_param_all_test.to_csv(f'{TEST_RESULT_DIR}/{identifier}_predictions.csv')

In [ ]:
from scipy.stats import spearmanr
import seaborn as sns

def subplot_recovery_by_trials(data: pd.DataFrame, label: str, trained_trial: str):
  true_l, dl_l = f'true_{label}', f'dl_{label}'

  g = sns.lmplot(x =true_l, y =dl_l,
            col = "num_test_trial", col_wrap=3,
            line_kws = {"color": "red"},
            data = data)
  g.set_axis_labels(f"True {label}", f"Predicted {label}").set_titles("Trials: {col_name}")

  for ax, feature in zip(g.axes.flat, g.col_names):
    param_all = data.loc[data.num_test_trial == feature]
    r_value, p_value = spearmanr(param_all[true_l], param_all[dl_l])
    ax.set_title(ax.get_title() + f', r={r_value:.2f}')

  g.fig.suptitle(f'{trained_trial}t GRU {label} recovery', fontsize='xx-large')
  plt.tight_layout()
  #plt.savefig(f'{TEST_RESULT_DIR}/{trained_trial}t_{label}_recovery.png')



for l in test_name_to_labels.keys():
  subplot_recovery_by_trials(trial_to_param_all_test, l, target_trial)

In [ ]:
for t in all_trials:
  trial_to_param_all_test = pd.read_csv(f'{TEST_RESULT_DIR}/{t}t_predictions.csv')
  for l in test_name_to_labels.keys():
    subplot_recovery_by_trials(trial_to_param_all_test, l, t)

In [ ]:
# @title single parameter recovery plot

label = list(train_name_to_labels.keys())[0]
p = plot_recovery(trial_to_param_all_test.loc[trial_to_param_all_test.num_test_trial == 500], label)
#p.draw()
fig = p.draw()
fig.savefig(f"{TEST_RESULT_DIR}/{dist_name}_recover_{label}.png")

In [ ]:
def build_parameters(normalized_test_labels, prediction, sorted_lables):
  from collections import defaultdict

  param_all_test = {}
  for idx in range(normalized_test_labels.shape[1]):
    l = sorted_lables[idx]
    k = f'true_{l}'
    param_all_test[k] = normalized_test_labels[:, idx]

    k = f'dl_{l}'
    param_all_test[k] = prediction[:, idx]

  return pd.DataFrame(param_all_test)

normalized_param_all_test = build_parameters(normalized_test_labels, prediction, ['alpha', 'beta'])
#normalized_param_all_test.to_csv(f"{TEST_RESULT_DIR}/{identifier}_predict_normalized.csv")

## GridSearch

In [ ]:
# For grid search
!pip install --upgrade keras-hypetune

In [ ]:
from kerashypetune import KerasBayesianSearch
from hyperopt import hp, Trials

if model_type == 'Transformer':
  model_func = get_transformer_model
  param_grid = {
      'input_shape': train_features.shape[1:],
      'head_size': 4 + hp.randint('head_size', 256),
      'num_head': 2 + hp.randint('num_head', 2),
      'ff_dim': 4,
      'mlp_units': 64, # + hp.randint('mlp_units', 256),
      'dropout': hp.uniform('dropout', .1, .4),
      'mlp_dropout': hp.uniform('mlp_dropout', .1, .4),
      'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.25)),
      'epochs': 20,
      'num_transformer_blocks': 1 + hp.randint('num_transformer_blocks', 4),
      'batch_size': 256,
  }
else:
  model_func = get_gru_model
  param_grid = {
    'input_x': train_features.shape[1],
    'input_y': train_features.shape[2],
    'units': 64 + hp.randint('units', 128),
    'learning_rate': 0.0079, #hp.loguniform('learning_rate', np.log(3e-4), np.log(0.2)),
    'dropout': 0.2, #hp.uniform('dropout', .15, .25),
    'dropout1': 0.02, #hp.uniform('dropout1', .01, .1),
    'dropout2': 0.02, #hp.uniform('dropout2', .01, .05),
    'epochs': 30,
    'batch_size': 256,
  }

kbs = KerasBayesianSearch(model_func, param_grid, monitor='val_loss', greater_is_better=False, n_iter=10, sampling_seed=11)
callbacks = [EarlyStopping(monitor='val_loss', patience=20)]
kbs.search(train_features, normalized_train_labels, trials=Trials(), validation_data=(val_features, normalized_val_labels), callbacks=callbacks)
print(kbs.best_params)
print(kbs.scores)

In [ ]:
from kerashypetune import KerasGridSearch

# Set up hyperparameters to test
param_grid = {
    'input_x': [train_features.shape[1]],
    'input_y': [train_features.shape[2]],
    'units': [128, 96],
    'dropout': [0.2, 0.1],
    'dropout1': [0.1, 0.05],
    'dropout2': [0.01, 0.05],
    'learning_rate': 0.0085, #3e-4,
    'epochs': 25,
    'batch_size': 512,
}


kgs = KerasGridSearch(get_gru_model, param_grid, monitor='val_loss', greater_is_better=False)
callbacks = [EarlyStopping(monitor='val_loss', patience=15)]
kgs.search(train_features, normalized_train_labels, validation_data=(val_features, normalized_val_labels), callbacks=callbacks)
print(kgs.best_params)
print(kgs.scores)